In [32]:
%config IPCompleter.greedy=True



In [33]:
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx

from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.aqua import Operator, run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import max_cut, tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

In [34]:

a = -5
b = -3
#xRaw = 4
#yRaw = 5

def getBit(number,precision,posicion):
    return int(format(number,'b').rjust(precision,'0')[precision-1-i])

In [35]:
n = 3
from docplex.mp.model import Model
from qiskit.aqua.translators.ising import docplex

# Create an instance of a model and variables.
mdl = Model(name='max_cut')
x = {i: mdl.binary_var(name='x_{0}'.format(i)) for i in range(n)}
y = {i: mdl.binary_var(name='y_{0}'.format(i)) for i in range(n)}

# Object function
precision = 4
max_cut_func = mdl.sum(a*2**i*x[i]+b*2**i*y[i] for i in range(n))

mdl.maximize(max_cut_func)


# Magic

In [36]:
qubitOp_docplex, offset_docplex = docplex.get_qubitops(mdl)


# More Magic

In [37]:
seed = 10598

spsa = SPSA(max_trials=300)
ry = RY(qubitOp_docplex.num_qubits, depth=4, entanglement='linear')
vqe = VQE(qubitOp_docplex, ry, spsa, 'matrix')

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)
x = max_cut.sample_most_likely(result['eigvecs'][0])
print('solution:', max_cut.get_graph_solution(x))
#print('solution objective:', max_cut.max_cut_value(x, w))

solution: [1. 1. 1. 1. 1. 1.]
